In [ ]:
! pip install -q --upgrade langchain langchain-openai langchain-core langchain_community docx2txt pypdf  langchain_chroma sentence_transformers

### **Part 1:**

- **Understanding RAG (Retrieval-Augmented Generation)**
    - What is RAG?
    - Importance of RAG in AI applications
- **Introduction to LangChain and LCEL**
    - Overview of LangChain
    - LangChain Expression Language (LCEL)
- **Exploring LangChain Components**
    - LLM (Large Language Models)
    - Prompts
    - Retrievers
    - Composing components into a chain
- **Document Processing and Vector Databases**
    - Document splitting techniques
    - Embedding documents
    - Storing and retrieving documents in a vector database

- **Creating Your First RAG Chain**
    - Step-by-step guide to building a basic RAG chain
    - Answering questions from documents using RAG
- **Building a Conversational RAG**
    - Handling follow-up questions
    - Concept of contextualizing and refining queries
- **Using pre-built LangChain RAG chains**
    - history_aware_retriever
    - create_retrieval_chain
- **Building Multi User Chatbot**
    - Managing conversation history using a database table

### **Part 2: Moving to Production with FastAPI**

- **Integrating Colab Code with FastAPI**
    - Setting up FastAPI for production
    - Modularizing code into different files for maintainability
    - Implemeting chatbot endpoint to talk to your data
- **Creating API Endpoints**
    - Building endpoints for file upload, list, and deletion
    - Testing and validating endpoints

- **Building a Streamlit Interface**
    - Creating a user-friendly interface with Streamlit
    - Integrating the RAG chatbot API with the Streamlit app
    - File management features (upload, list, delete)

###What is Retrieval Augmented Generation (RAG)?
RAG is a technique that enhances language models by combining them with a retrieval system. It allows the model to access and utilize external knowledge when generating responses.

The process typically involves:
####Indexing a large corpus of documents

In [ ]:
import langchain
print(langchain.__version__)

0.3.4


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_PROJECT"] = "langchain-course"

###Call LLM

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")
llm_response = llm.invoke("Tell me a joke")

llm_response

AIMessage(content='Why did the scarecrow win an award?\n\nBecause he was outstanding in his field!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 11, 'total_tokens': 28, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_482c22a7bc', 'finish_reason': 'stop', 'logprobs': None}, id='run-78cb68a5-3613-4edb-bc3e-ff9f4b211680-0', usage_metadata={'input_tokens': 11, 'output_tokens': 17, 'total_tokens': 28, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

###Parsing Output

In [ ]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
output_parser.invoke(llm_response)

'Why did the scarecrow win an award?\n\nBecause he was outstanding in his field!'

###Simple Chain

In [ ]:
chain = llm | output_parser
chain.invoke("Tell me a joke")

'Why did the scarecrow win an award?\n\nBecause he was outstanding in his field!'

###Structured Output

In [ ]:
from typing import List
from pydantic import BaseModel, Field

class MobileReview(BaseModel):
    phone_model: str = Field(description="Name and model of the phone")
    rating: float = Field(description="Overall rating out of 5")
    pros: List[str] = Field(description="List of positive aspects")
    cons: List[str] = Field(description="List of negative aspects")
    summary: str = Field(description="Brief summary of the review")

review_text = """
Just got my hands on the new Galaxy S21 and wow, this thing is slick! The screen is gorgeous,
colors pop like crazy. Camera's insane too, especially at night - my Insta game's never been
stronger. Battery life's solid, lasts me all day no problem.

Not gonna lie though, it's pretty pricey. And what's with ditching the charger? C'mon Samsung.
Also, still getting used to the new button layout, keep hitting Bixby by mistake.

Overall, I'd say it's a solid 4 out of 5. Great phone, but a few annoying quirks keep it from
being perfect. If you're due for an upgrade, definitely worth checking out!
"""

structured_llm = llm.with_structured_output(MobileReview)
output = structured_llm.invoke(review_text)
output

MobileReview(phone_model='Samsung Galaxy S21', rating=4.0, pros=['Gorgeous screen', 'Vibrant colors', 'Insane camera quality, especially at night', 'Solid battery life'], cons=['Pricey', 'No charger included', 'New button layout can be confusing'], summary="Overall, it's a solid 4 out of 5. Great phone, but a few annoying quirks keep it from being perfect. If you're due for an upgrade, definitely worth checking out!")

In [ ]:
output.pros

['Gorgeous screen',
 'Vibrant colors',
 'Insane camera quality, especially at night',
 'Solid battery life']

###Prompt Template

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")
prompt.invoke({"topic": "programming"})

ChatPromptValue(messages=[HumanMessage(content='Tell me a short joke about programming', additional_kwargs={}, response_metadata={})])

In [ ]:
chain = prompt | llm | output_parser
chain.invoke({"topic": "programer"})

'Why do programmers prefer dark mode?\n\nBecause light attracts bugs!'

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Define the prompt
prompt = ChatPromptTemplate.from_template("Tell me a short joke about {topic}")

# Initialize the LLM
llm = ChatOpenAI(model="gpt-4o-mini")

# Define the output parser
output_parser = StrOutputParser()

# Compose the chain
chain = prompt | llm | output_parser

# Use the chain
result = chain.invoke({"topic": "programming"})
print(result)

Why do programmers prefer dark mode?

Because light attracts bugs!


###LLM Messages

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import HumanMessage, SystemMessage

system_message = SystemMessage(content="You are a helpful assistant that tells jokes.")
human_message = HumanMessage(content="Tell me about programming")
llm.invoke([system_message, human_message])

AIMessage(content="Sure! Programming is the process of creating a set of instructions that a computer can follow to perform specific tasks. It involves writing code in various programming languages like Python, Java, C++, and many others. \n\nAnd speaking of programming, here's a joke for you:\n\nWhy do programmers prefer dark mode?\n\nBecause light attracts bugs!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 65, 'prompt_tokens': 24, 'total_tokens': 89, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_482c22a7bc', 'finish_reason': 'stop', 'logprobs': None}, id='run-e8b9a539-9bdb-47d2-8120-4684bbdf258e-0', usage_metadata={'input_tokens': 24, 'output_tokens': 65, 'total_tokens': 89, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [ ]:
template = ChatPromptTemplate([
    ("system", "You are a helpful assistant that tells jokes."),
    ("human", "Tell me about {topic}")
])

prompt_value = template.invoke(
    {
        "topic": "programming"
    }
)
prompt_value

ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant that tells jokes.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Tell me about programming', additional_kwargs={}, response_metadata={})])

In [ ]:
llm.invoke(prompt_value)

AIMessage(content='Sure! Here’s a light-hearted joke about programming:\n\nWhy do programmers prefer dark mode?\n\nBecause light attracts bugs! \n\nIf you want to know more about programming concepts or languages, feel free to ask!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 24, 'total_tokens': 65, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_482c22a7bc', 'finish_reason': 'stop', 'logprobs': None}, id='run-808f0c38-920d-4d5a-99fb-de6f0765b96d-0', usage_metadata={'input_tokens': 24, 'output_tokens': 41, 'total_tokens': 65, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [ ]:
# !pip install docx2txt pypdf unstructured

In [ ]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from typing import List
from langchain_core.documents import Document

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)

docx_loader = Docx2txtLoader("/content/docs/GreenGrow Innovations_ Company History.docx")
documents = docx_loader.load()

print(len(documents))

splits = text_splitter.split_documents(documents)

print(f"Split the documents into {len(splits)} chunks.")

1
Split the documents into 2 chunks.


In [ ]:
documents[0]

Document(metadata={'source': '/content/docs/GreenGrow Innovations_ Company History.docx'}, page_content="GreenGrow Innovations was founded in 2010 by Sarah Chen and Michael Rodriguez, two agricultural engineers with a passion for sustainable farming. The company started in a small garage in Portland, Oregon, with a simple mission: to make farming more environmentally friendly and efficient.\n\n\n\nIn its early days, GreenGrow focused on developing smart irrigation systems that could significantly reduce water usage in agriculture. Their first product, the WaterWise Sensor, was launched in 2012 and quickly gained popularity among local farmers. This success allowed the company to expand its research and development efforts.\n\n\n\nBy 2015, GreenGrow had outgrown its garage origins and moved into a proper office and research facility in the outskirts of Portland. This move coincided with the development of their second major product, the SoilHealth Monitor, which used advanced sensors to

In [ ]:
splits[1]

Document(metadata={'source': '/content/docs/GreenGrow Innovations_ Company History.docx'}, page_content="The company's breakthrough came in 2018 with the introduction of the EcoHarvest System, an integrated solution that combined smart irrigation, soil monitoring, and automated harvesting techniques. This system caught the attention of large-scale farmers across the United States, propelling GreenGrow to national prominence.\n\n\n\nToday, GreenGrow Innovations employs over 200 people and has expanded its operations to include offices in California and Iowa. The company continues to focus on developing sustainable agricultural technologies, with ongoing projects in vertical farming, drought-resistant crop development, and AI-powered farm management systems.\n\n\n\nDespite its growth, GreenGrow remains committed to its original mission of promoting sustainable farming practices. The company regularly partners with universities and research institutions to advance the field of agricultura

In [ ]:
splits[0].metadata

{'source': '/content/docs/GreenGrow Innovations_ Company History.docx'}

In [ ]:
splits[0].page_content

'GreenGrow Innovations was founded in 2010 by Sarah Chen and Michael Rodriguez, two agricultural engineers with a passion for sustainable farming. The company started in a small garage in Portland, Oregon, with a simple mission: to make farming more environmentally friendly and efficient.\n\n\n\nIn its early days, GreenGrow focused on developing smart irrigation systems that could significantly reduce water usage in agriculture. Their first product, the WaterWise Sensor, was launched in 2012 and quickly gained popularity among local farmers. This success allowed the company to expand its research and development efforts.\n\n\n\nBy 2015, GreenGrow had outgrown its garage origins and moved into a proper office and research facility in the outskirts of Portland. This move coincided with the development of their second major product, the SoilHealth Monitor, which used advanced sensors to analyze soil composition and provide real-time recommendations for optimal crop growth.'

In [ ]:
# import nltk
# nltk.download('punkt')

In [ ]:
# 1. Function to load documents from a folder

def load_documents(folder_path: str) -> List[Document]:
    documents = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif filename.endswith('.docx'):
            loader = Docx2txtLoader(file_path)
        else:
            print(f"Unsupported file type: {filename}")
            continue
        documents.extend(loader.load())
    return documents

# Load documents from a folder
folder_path = "/content/docs"
documents = load_documents(folder_path)

print(f"Loaded {len(documents)} documents from the folder.")
splits = text_splitter.split_documents(documents)
print(f"Split the documents into {len(splits)} chunks.")

Loaded 5 documents from the folder.
Split the documents into 8 chunks.


In [ ]:
embeddings = OpenAIEmbeddings()

# 4. Embedding Documents

document_embeddings = embeddings.embed_documents([split.page_content for split in splits])

print(f"Created embeddings for {len(document_embeddings)} document chunks.")

Created embeddings for 8 document chunks.


In [ ]:
document_embeddings[0]

In [ ]:
# !pip install sentence_transformers
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
document_embeddings = embedding_function.embed_documents([split.page_content for split in splits])
document_embeddings[0]

In [ ]:
# !pip install langchain_chroma -q

In [ ]:
# import shutil

# shutil.rmtree('chroma_db')

###Create and persist Chroma vector store

In [ ]:
from langchain_chroma import Chroma

embedding_function = OpenAIEmbeddings()
collection_name = "my_collection"
vectorstore = Chroma.from_documents(collection_name=collection_name, documents=splits, embedding=embedding_function, persist_directory="./chroma_db")
#db.persist()

print("Vector store created and persisted to './chroma_db'")

Vector store created and persisted to './chroma_db'


In [ ]:
# 5. Perform similarity search

query = "When was GreenGrow Innovations founded?"
search_results = vectorstore.similarity_search(query, k=2)

print(f"\nTop 2 most relevant chunks for the query: '{query}'\n")
for i, result in enumerate(search_results, 1):
    print(f"Result {i}:")
    print(f"Source: {result.metadata.get('source', 'Unknown')}")
    print(f"Content: {result.page_content}")
    print()


Top 2 most relevant chunks for the query: 'When was GreenGrow Innovations founded?'

Result 1:
Source: /content/docs/GreenGrow Innovations_ Company History.docx
Content: The company's breakthrough came in 2018 with the introduction of the EcoHarvest System, an integrated solution that combined smart irrigation, soil monitoring, and automated harvesting techniques. This system caught the attention of large-scale farmers across the United States, propelling GreenGrow to national prominence.



Today, GreenGrow Innovations employs over 200 people and has expanded its operations to include offices in California and Iowa. The company continues to focus on developing sustainable agricultural technologies, with ongoing projects in vertical farming, drought-resistant crop development, and AI-powered farm management systems.



Despite its growth, GreenGrow remains committed to its original mission of promoting sustainable farming practices. The company regularly partners with universities and

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
retriever.invoke("When was GreenGrow Innovations founded?")

[Document(metadata={'source': '/content/docs/GreenGrow Innovations_ Company History.docx'}, page_content="The company's breakthrough came in 2018 with the introduction of the EcoHarvest System, an integrated solution that combined smart irrigation, soil monitoring, and automated harvesting techniques. This system caught the attention of large-scale farmers across the United States, propelling GreenGrow to national prominence.\n\n\n\nToday, GreenGrow Innovations employs over 200 people and has expanded its operations to include offices in California and Iowa. The company continues to focus on developing sustainable agricultural technologies, with ongoing projects in vertical farming, drought-resistant crop development, and AI-powered farm management systems.\n\n\n\nDespite its growth, GreenGrow remains committed to its original mission of promoting sustainable farming practices. The company regularly partners with universities and research institutions to advance the field of agricultur

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer: """
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()} | prompt
)
rag_chain.invoke("When was GreenGrow Innovations founded?")

ChatPromptValue(messages=[HumanMessage(content='Answer the question based only on the following context:\n[Document(metadata={\'source\': \'/content/docs/GreenGrow Innovations_ Company History.docx\'}, page_content="The company\'s breakthrough came in 2018 with the introduction of the EcoHarvest System, an integrated solution that combined smart irrigation, soil monitoring, and automated harvesting techniques. This system caught the attention of large-scale farmers across the United States, propelling GreenGrow to national prominence.\\n\\n\\n\\nToday, GreenGrow Innovations employs over 200 people and has expanded its operations to include offices in California and Iowa. The company continues to focus on developing sustainable agricultural technologies, with ongoing projects in vertical farming, drought-resistant crop development, and AI-powered farm management systems.\\n\\n\\n\\nDespite its growth, GreenGrow remains committed to its original mission of promoting sustainable farming p

In [ ]:
def docs2str(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
rag_chain = (
    {"context": retriever | docs2str, "question": RunnablePassthrough()} | prompt
)
rag_chain.invoke("When was GreenGrow Innovations founded?")

ChatPromptValue(messages=[HumanMessage(content="Answer the question based only on the following context:\nThe company's breakthrough came in 2018 with the introduction of the EcoHarvest System, an integrated solution that combined smart irrigation, soil monitoring, and automated harvesting techniques. This system caught the attention of large-scale farmers across the United States, propelling GreenGrow to national prominence.\n\n\n\nToday, GreenGrow Innovations employs over 200 people and has expanded its operations to include offices in California and Iowa. The company continues to focus on developing sustainable agricultural technologies, with ongoing projects in vertical farming, drought-resistant crop development, and AI-powered farm management systems.\n\n\n\nDespite its growth, GreenGrow remains committed to its original mission of promoting sustainable farming practices. The company regularly partners with universities and research institutions to advance the field of agricultur

In [ ]:
rag_chain = (
    {"context": retriever | docs2str, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
question = "When was GreenGrow Innovations founded?"
response = rag_chain.invoke(question)
print(response)

GreenGrow Innovations was founded in 2010.


###Conversational RAG

####Handling Follow Up Questions

In [ ]:
# Example conversation
from langchain_core.messages import HumanMessage, AIMessage
chat_history = []
chat_history.extend([
    HumanMessage(content=question),
    AIMessage(content=response)
])

In [ ]:
chat_history

[HumanMessage(content='When was GreenGrow Innovations founded?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='GreenGrow Innovations was founded in 2010.', additional_kwargs={}, response_metadata={})]

In [ ]:
from langchain_core.prompts import MessagesPlaceholder
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# history_aware_retriever = create_history_aware_retriever(
#     llm, retriever, contextualize_q_prompt
# )
contextualize_chain = contextualize_q_prompt | llm | StrOutputParser()
contextualize_chain.invoke({"input": "Where it is headquartered?", "chat_history": chat_history})

'Where is GreenGrow Innovations headquartered?'

In [ ]:
from langchain.chains import create_history_aware_retriever
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)
history_aware_retriever.invoke({"input": "Where it is headquartered?", "chat_history": chat_history})

[Document(metadata={'source': '/content/docs/GreenGrow Innovations_ Company History.docx'}, page_content="The company's breakthrough came in 2018 with the introduction of the EcoHarvest System, an integrated solution that combined smart irrigation, soil monitoring, and automated harvesting techniques. This system caught the attention of large-scale farmers across the United States, propelling GreenGrow to national prominence.\n\n\n\nToday, GreenGrow Innovations employs over 200 people and has expanded its operations to include offices in California and Iowa. The company continues to focus on developing sustainable agricultural technologies, with ongoing projects in vertical farming, drought-resistant crop development, and AI-powered farm management systems.\n\n\n\nDespite its growth, GreenGrow remains committed to its original mission of promoting sustainable farming practices. The company regularly partners with universities and research institutions to advance the field of agricultur

In [ ]:
retriever.invoke("Where it is headquartered?")

[Document(metadata={'source': '/content/docs/Company_ QuantumNext Systems.docx'}, page_content='Company: QuantumNext Systems\n\nHeadquarters: QuantumNext Systems is headquartered in Bangalore, Karnataka, India. The company, specializing in quantum computing and advanced data processing, is situated in the bustling tech metropolis of Bangalore, often referred to as the "Silicon Valley of India." From this technology capital, QuantumNext Systems is well-positioned to tap into India\'s rich pool of engineering talent and growing tech ecosystem, enabling it to push the boundaries of computational innovation.'),
 Document(metadata={'source': '/content/docs/Company_ TechWave Innovations.docx'}, page_content='Company: TechWave Innovations\n\nHeadquarters: TechWave Innovations is headquartered in San Francisco, California, USA. As a leader in cutting-edge AI and machine learning solutions, the company thrives in the heart of Silicon Valley, benefiting from its proximity to tech giants and a dy

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the following context to answer the user's question."),
    #  ("system", "Tell me joke on Programming"),
    ("system", "Context: {context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [ ]:
rag_chain.invoke({"input": "Where it is headquartered?", "chat_history":chat_history})

{'input': 'Where it is headquartered?',
 'chat_history': [HumanMessage(content='When was GreenGrow Innovations founded?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='GreenGrow Innovations was founded in 2010.', additional_kwargs={}, response_metadata={})],
 'context': [Document(metadata={'source': '/content/docs/GreenGrow Innovations_ Company History.docx'}, page_content="The company's breakthrough came in 2018 with the introduction of the EcoHarvest System, an integrated solution that combined smart irrigation, soil monitoring, and automated harvesting techniques. This system caught the attention of large-scale farmers across the United States, propelling GreenGrow to national prominence.\n\n\n\nToday, GreenGrow Innovations employs over 200 people and has expanded its operations to include offices in California and Iowa. The company continues to focus on developing sustainable agricultural technologies, with ongoing projects in vertical farming, drought-resistant

###Building Multi User Chatbot

In [ ]:
import sqlite3
from datetime import datetime

DB_NAME = "rag_app.db"

def get_db_connection():
    conn = sqlite3.connect(DB_NAME)
    conn.row_factory = sqlite3.Row
    return conn

def create_application_logs():
    conn = get_db_connection()
    conn.execute('''CREATE TABLE IF NOT EXISTS application_logs
                    (id INTEGER PRIMARY KEY AUTOINCREMENT,
                     session_id TEXT,
                     user_query TEXT,
                     gpt_response TEXT,
                     model TEXT,
                     created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP)''')
    conn.close()

def insert_application_logs(session_id, user_query, gpt_response, model):
    conn = get_db_connection()
    conn.execute('INSERT INTO application_logs (session_id, user_query, gpt_response, model) VALUES (?, ?, ?, ?)',
                 (session_id, user_query, gpt_response, model))
    conn.commit()
    conn.close()

def get_chat_history(session_id):
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute('SELECT user_query, gpt_response FROM application_logs WHERE session_id = ? ORDER BY created_at', (session_id,))
    messages = []
    for row in cursor.fetchall():
        messages.extend([
            {"role": "human", "content": row['user_query']},
            {"role": "ai", "content": row['gpt_response']}
        ])
    conn.close()
    return messages

# Initialize the database
create_application_logs()

In [ ]:
import uuid
session_id = str(uuid.uuid4())
chat_history = get_chat_history(session_id)
print(chat_history)
question1 = "When was GreenGrow Innovations founded?"
answer1 = rag_chain.invoke({"input": question1, "chat_history":chat_history})['answer']
insert_application_logs(session_id, question1, answer1, "gpt-4-o-mini")
print(f"Human: {question1}")
print(f"AI: {answer1}\n")

[]
Human: When was GreenGrow Innovations founded?
AI: GreenGrow Innovations was founded in 2010.



In [ ]:
question2 = "Where it is headquartered?"
chat_history = get_chat_history(session_id)
print(chat_history)
answer2 = rag_chain.invoke({"input": question2, "chat_history":chat_history})['answer']
insert_application_logs(session_id, question2, answer2, "gpt-3.5-turbo")
print(f"Human: {question2}")
print(f"AI: {answer2}\n")

[{'role': 'human', 'content': 'When was GreenGrow Innovations founded?'}, {'role': 'ai', 'content': 'GreenGrow Innovations was founded in 2010.'}]
Human: Where it is headquartered?
AI: GreenGrow Innovations is headquartered in Portland, Oregon. Additionally, the company has expanded its operations to include offices in California and Iowa.



New User

In [ ]:
session_id = str(uuid.uuid4())
question = "What is GreenGrow"
chat_history = get_chat_history(session_id)
print(chat_history)
answer = rag_chain.invoke({"input": question, "chat_history":chat_history})['answer']
insert_application_logs(session_id, question, answer, "gpt-3.5-turbo")
print(f"Human: {question}")
print(f"AI: {answer}\n")